In [ ]:
import cv2
import numpy as np
import math

USE_LINE_INSECT = 3 # [昆虫画像256/line切り替え設定] 2: AIの線画 0: 256昆虫画像を使用する 1: line昆虫画像を使用する
DISP_DOG_WINDOW = 0 # [犬画像の表示設定] 0:最終画像以外の表示なし 1: 初期ロード直後の画像も表示 2: さらに、輪郭抽出時の加工画像等も表示
BUNKATSU = 4 #中間層の分割数

#
# モード設定変数(輪郭配置関連)
#
PLACE_EDGE_INSECTS = 1 # 0:輪郭線沿いに昆虫を表示しない 1:輪郭線沿いに昆虫を表示する
DRAW_EDGE_FIGURE = 0 # 1:昆虫画像位置ガイドとしての直線・楕円を描画する  0:昆虫画像位置ガイドとしての直線・楕円を描画しない
DISP_EDGE_INSECT_WINDOW = 0 # [別ウィンドウでの昆虫の表示] 0:表示なし 1: アフィン変換後の画像を表示 2: さらに、初期ロード直後の画像も表示

GREEN = (0, 255, 0)
THCK = 1

'''insect_images_AI_c_new = [
    #'mantis4'
    'mantis10','mantis11','mantis12','mantis13','mantis14','mantis15','mantis16','mantis17','mantis18','mantis19',
    'mantis110','mantis111','mantis112','mantis113','mantis114','mantis115','mantis116','mantis117','mantis118','mantis119',
    'mantis120','mantis121','mantis122','mantis123','mantis124','mantis125','mantis126','mantis127',
]'''
insect_images_AI_c_new = [
    'hornet6'
    #'hornet60','hornet61','hornet62','hornet63','hornet64','hornet65','hornet66','hornet67','hornet68','hornet69',
    #'hornet610','hornet611','hornet612','hornet613','hornet614','hornet615','hornet616','hornet617','hornet618','hornet619',
    #'hornet620','hornet621','hornet622','hornet623','hornet624',
]

# 昆虫画像の胸部と腹部の分割基準点を利用して輪郭線中点に合わせる

def calc_midpoint(start_p, end_p):
    return (start_p[0] + end_p[0]) // 2, (start_p[1] + end_p[1]) // 2

def calc_distance(start_p, end_p):
    delta_x, delta_y = calc_delta(start_p, end_p)
    return int(math.sqrt(delta_x**2 + delta_y**2))

def calc_delta(start_p, end_p):
    delta_x = end_p[0] - start_p[0]
    delta_y = end_p[1] - start_p[1]

    return delta_x, delta_y

def rotate_and_place_image(insect_img, back_img, center_point, angle, scale):
    h, w = insect_img.shape[:2]
    center = (w // 2, h // 2)

    # 回転変換行列の算出
    rot_mtx = cv2.getRotationMatrix2D(center, angle, scale)

    # アフィン変換
    rotated_img = cv2.warpAffine(insect_img, rot_mtx, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

    # 合成処理
    process_image(rotated_img, back_img, center_point[0], center_point[1])
    
def rotate_image(img, angle, center=None, scale=1.0):
    (h, w) = img.shape[:2]
    
    if center is None:
        center = (w // 2, h // 2)
        
    # 回転行列を生成
    rot_mat = cv2.getRotationMatrix2D(center, angle, scale)
    rotated_img = cv2.warpAffine(img, rot_mat, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))
    
    return rotated_img

def calc_angle(start_p, end_p):
    delta_x, delta_y = calc_delta(start_p, end_p)
    angle_radians = math.atan2(delta_y, delta_x)
    angle_degrees = math.degrees(angle_radians)
    return (angle_degrees + 90) % 360

def get_largest_contour(img):

    # 画像をグレースケールに変換
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 画像を二値化
    ret, img_bin = cv2.threshold(img_gray, 127, 255,0)

    # 画像輪郭抽出
    contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #   
    # rint('num of contours=' + str(len(contours)))

    # 最大輪郭決定
    largest_contour = []
    num_points = 0
    for cnt in contours:
        if len(cnt) > num_points:
            num_points = len(cnt)
            largest_contour = cnt

    return largest_contour

def split_points(start_point, end_point):
    # 1/2 と 1/2 の位置で分割
    mid_point1 = [start_point[0] + (end_point[0] - start_point[0]) / 2, 
                  start_point[1] + (end_point[1] - start_point[1]) / 2]
    
    mid_point2 = [start_point[0] + 2 * (end_point[0] - start_point[0]) / 2, 
                  start_point[1] + 2 * (end_point[1] - start_point[1]) / 2]
    
    return mid_point1, mid_point2

def calc_angle_from_points(start_point, end_point):
    # 座標の差を求める
    dx = end_point[0] - start_point[0]
    dy = end_point[1] - start_point[1]
    
    # atan2関数を使って角度を計算（ラジアン）
    angle_rad = math.atan2(dy, dx)
    
    # ラジアンを度に変換
    angle_deg = math.degrees(angle_rad)
    
    #小数点以下切り捨て
    angle_deg_t = math.floor(angle_deg)

    #if angle_deg_t < 0:
    #    angle_deg_t += 360
    
    return angle_deg_t

def process_image(img_insect, img_back, midpoint_x, midpoint_y):
    # サイズ情報を取得
    insect_height, insect_width = img_insect.shape[:2]
    back_height, back_width = img_back.shape[:2]

    # 配置位置計算
    y_offset = max(0, midpoint_y - insect_height // 2)
    x_offset = max(0, midpoint_x - insect_width // 2)

    y_reduce_upper = max(0, - (midpoint_y - insect_height // 2))
    x_reduce_left = max(0, - (midpoint_x - insect_width // 2))

    y2 = min(back_height, y_offset + insect_height - y_reduce_upper)
    x2 = min(back_width, x_offset + insect_width - x_reduce_left)

    # 背景画像に収まる範囲を計算
    y1, x1 = y_offset, x_offset
    insect_roi = img_insect[y_reduce_upper:(y2 - y_offset + y_reduce_upper),
                            x_reduce_left:(x2 - x_offset + x_reduce_left)]

    # アルファチャンネルの分離
    alpha_insect = insect_roi[:, :, 3] / 255.0
    alpha_back = 1.0 - alpha_insect

    # BGRチャンネルごとに合成
    for c in range(3):
        img_back[y1:y2, x1:x2, c] = (
            alpha_insect * insect_roi[:, :, c] +
            alpha_back * img_back[y1:y2, x1:x2, c]
        )



def place_insect_along_line(back_img, insect_start_point, insect_end_point, idx, draw_figure):
    """
    輪郭線上に昆虫画像を配置する。
    """
    # 2点を結ぶ直線を描画
    if draw_figure == 1:
        cv2.line(back_img, pt1=insect_start_point, pt2=insect_end_point, color=GREEN, thickness=THCK)

    # 楕円を描画 
    mp_x, mp_y = calc_midpoint(insect_start_point, insect_end_point)
    dist = calc_distance(insect_start_point, insect_end_point)
    angle = calc_angle(insect_start_point, insect_end_point)
    # 内側に平行移動した楕円を描画################################試
    delta_x = insect_end_point[0] - insect_start_point[0]
    delta_y = insect_end_point[1] - insect_start_point[1]
    
    direction_x = delta_y / dist
    direction_y = (-1) * delta_x / dist

    mp_x_moved = int(mp_x + direction_x * (dist//4))
    mp_y_moved = int(mp_y + direction_y * (dist//4))
    mp_x_moved_2 = int(mp_x + direction_x * (dist//4)/4)
    mp_y_moved_2 = int(mp_y + direction_y * (dist//4)/3)

    # 昆虫画像のロード
    if USE_LINE_INSECT == 3:
        insect_file_path = './images/wakuta_c2/' + insect_images_AI_c_new[idx % len(insect_images_AI_c_new)] + '_AI.png'
    else:
        return  # 画像が見つからない場合は何もしない

    insect_img = cv2.imread(insect_file_path, cv2.IMREAD_UNCHANGED)
    if insect_img is None:
        return

    # ===============================================================
    # αチャネル付き昆虫画像をアフィン変換(回転/拡縮の実行)
    # ===============================================================
    insect_height, insect_width = (insect_img.shape[0] , insect_img.shape[1])
    scale = dist / insect_height * 1.49	# 画像内の余白部を考慮し、拡大する

    insect_size = (insect_width, insect_height)
    center = (insect_width / 2, insect_height / 2)

    angle_mod = (-angle + 544) % 360
    # 回転変換行列の算出
    rot_mtx = cv2.getRotationMatrix2D(center, angle_mod, scale)

    # アフィン変換
    insect_img_rot = cv2.warpAffine(insect_img, rot_mtx, insect_size, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    # 合成処理#############要改善
    process_image(insect_img_rot, back_img, mp_x_moved_2, mp_y_moved_2)
    
    
def main():
    # 元画像と昆虫画像のロード
    img_back = cv2.imread('images/background/back_wakuta.jpg')
    img_mask = cv2.imread('images/background/wakuta_mask.png')
    #img_mask = cv2.imread('images/background/back_mask2.png')
    #img_mask = cv2.imread('images/background/horse2_mask.png')
    #img_mask = cv2.imread('images/background/rabbit_mask.png')
    #img_mask = cv2.imread('images/background/cat.png')
    #img_mask = cv2.imread('images/background/cat2.png')
    #img_mask = cv2.imread('images/background/cat3.png')
    

    # 輪郭取得
    contour_back = get_largest_contour(img_mask)
    points_outer = cv2.approxPolyDP(contour_back, 0.003 * cv2.arcLength(contour_back, True), True)

    copied_img = img_back.copy()

    # 輪郭直線毎昆虫配置処理
    if PLACE_EDGE_INSECTS == 1:

        draw_fig = (1 if (DRAW_EDGE_FIGURE == 1) else 0)	# ガイド図形描画有無

        # 昆虫画像の読み込み
        #insect_img = cv2.imread('images/wakuta_c2/mantis4_AI.png', cv2.IMREAD_UNCHANGED)
        insect_img = cv2.imread('images/wakuta_c2/hornet6_AI.png', cv2.IMREAD_UNCHANGED)

        
        if insect_img is not None:
            # 昆虫画像を上下に分割
            h, w = insect_img.shape[:2]
            chest_img = insect_img[:h//2, :]  # 胸部
            abdomen_img = insect_img[h//2:, :]  # 腹部
            
        for i in range(0, len(points_outer)):
            insect_start_point = [points_outer[i][0][0], points_outer[i][0][1]]
            j = (i+1) if (i+1) != len(points_outer) else 0
            insect_end_point = [points_outer[j][0][0], points_outer[j][0][1]]
            
            #start_pointとend_pointを1/2, 1/2で分割
            mid_point1,mid_point2 = split_points(insect_start_point, insect_end_point)
            
            #start_pointとmid_point1の角度を計算
            angle_start_mid = calc_angle_from_points(insect_start_point, mid_point1)
            #デバック用
            print(f"start_mid角度: {angle_start_mid} 度")
            
            if 0 <= angle_start_mid and angle_start_mid <= 55:
                angle_start_mid = 0
            else :
                angle_start_mid = -5

            #mid_point1とmid_point2の角度を計算
            angle_mid_end = calc_angle_from_points(mid_point1, mid_point2)
            angle_mid_end = 7
            #デバック用
            print(f"mid_end角度:   {angle_mid_end} 度")
            
            #回転
            rotated_chest = rotate_image(chest_img, angle_start_mid, (w//1.9 ,h//3))
            rotated_abdomen = rotate_image(abdomen_img, angle_mid_end, )
            
            # 合成画像の作成
            combined_height = rotated_chest.shape[0] + rotated_abdomen.shape[0]
            combined_width = max(rotated_chest.shape[1], rotated_abdomen.shape[1])
            combined_img = np.zeros((combined_height, combined_width, 4), dtype=np.uint8)  # 透明領域を含む合成画像

            # 胸部画像を合成
            combined_img[:rotated_chest.shape[0], :rotated_chest.shape[1]] = rotated_chest
            # 腹部画像を合成
            combined_img[rotated_chest.shape[0]:, :rotated_abdomen.shape[1]] = rotated_abdomen

            #
            #file_name = f'./images/wakuta_c2/mantis1{i}_AI.png'
            file_name = f'./images/wakuta_c2/hornet6{i}_AI.png'
            
            # 保存
            cv2.imwrite(file_name, combined_img)
            print("合成画像が保存されました。")
            print(i,"\n-----------------------")

            place_insect_along_line(copied_img, insect_start_point, insect_end_point, i, draw_fig)
        

    # 結果保存
    cv2.imwrite('./results/results_wakuta/wakuta16_result8.png', copied_img)

if __name__ == "__main__":
    main()


start_mid角度: 155 度
mid_end角度:   7 度
合成画像が保存されました。
0 
-----------------------
start_mid角度: 132 度
mid_end角度:   7 度
合成画像が保存されました。
1 
-----------------------
start_mid角度: 160 度
mid_end角度:   7 度
合成画像が保存されました。
2 
-----------------------
start_mid角度: 135 度
mid_end角度:   7 度
合成画像が保存されました。
3 
-----------------------
start_mid角度: 100 度
mid_end角度:   7 度
合成画像が保存されました。
4 
-----------------------
start_mid角度: 139 度
mid_end角度:   7 度
合成画像が保存されました。
5 
-----------------------
start_mid角度: 97 度
mid_end角度:   7 度
合成画像が保存されました。
6 
-----------------------
start_mid角度: 46 度
mid_end角度:   7 度
合成画像が保存されました。
7 
-----------------------
start_mid角度: 13 度
mid_end角度:   7 度
合成画像が保存されました。
8 
-----------------------
start_mid角度: -17 度
mid_end角度:   7 度
合成画像が保存されました。
9 
-----------------------
start_mid角度: -32 度
mid_end角度:   7 度
合成画像が保存されました。
10 
-----------------------
start_mid角度: 95 度
mid_end角度:   7 度
合成画像が保存されました。
11 
-----------------------
start_mid角度: 77 度
mid_end角度:   7 度
合成画像が保存されました。
12 
-----------------------
s